## <span style=color:blue>Patterns used in Programming Assignment 2 (version mostly avoiding the util.py file)  </span>

In [ ]:
# These are boiler plate imports that seem useful
# Perhaps cleaner would be to delete or comment out the ones that aren't used in this script...

import sys
import json
import csv
import yaml

import pandas as pd
import numpy as np

import datetime;

import matplotlib as mpl

import time
from datetime import datetime
# see https://stackoverflow.com/questions/415511/how-do-i-get-the-current-time-in-python
#   for some basics about datetime

import pprint

import os
from dotenv import load_dotenv

# sqlalchemy 2.0 documentation: https://www.sqlalchemy.org/
import psycopg2
from sqlalchemy import create_engine, text as sql_text

# the following is deprecated, it seems, so using the sqlalchemy
# from pyscopg2 import sqlio

# the file in benchmarking/util.py should hold utilities useful for your benchmarking exercise
# In this notebook we have commented out all mentions of util, so that you can run
#    this notebook before setting up your benchmarking/util.py file
# sys.path.append('benchmarking/')
# import util
# to invoke a function "foo()" inside util.py, use "util.foo()"

In [ ]:
dotenv_path = 'variables.env'
load_dotenv(dotenv_path=dotenv_path)

In [ ]:

load_dotenv()

schema = os.getenv('DISC_4_SCHEMA')
port = os.getenv('DISC_4_PORT')
host = os.getenv('DISC_4_HOST')
database = os.getenv('DISC_4_DB')
username = os.getenv('username')
password = os.getenv('password')

In [ ]:
# following https://www.geeksforgeeks.org/connecting-postgresql-with-sqlalchemy-in-python/

db_eng = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}",
                       connect_args={'options': '-csearch_path={}'.format(f"{schema}")},
                       isolation_level = 'SERIALIZABLE')
#    , echo=True)
#    , echo_pool="debug")

print("Successfully created db engine.")

# connect_args is used to set search_path to the schema 'new_york_city' in the airbnb database

# isolation_level SERIALIZABLE makes transactions happen in sequence, which is good 
#      for the benchmarking we will be doing

# for general info on sqlalchemy connections,
#    see: https://docs.sqlalchemy.org/en/20/core/connections.html

# echo from https://docs.sqlalchemy.org/en/20/core/engines.html

In [ ]:
sys.path.append('/Users/Nfaith21/Documents/ECS 116 - Misc/DISC_5_FILES/benchmarking/')
import util_main as util

In [ ]:
new_dict = {}

years = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
        '2020', '2021', '2022', '2023', '2024']
for year in years:
    new_dict['q_listings_join_reviews_'+year] = util.build_query_listings_join_reviews_datetime(year+'-01-01 12:00:00.000', year+'-12-31 12:00:00.000')


In [ ]:
all_perf_details = {}

all_indexes = [['datetime','reviews'], ['id','listings']] 


index_combinations = [
    [],
    [['datetime', 'reviews']],
    [['id', 'listings']],
    [['datetime', 'reviews'], ['id', 'listings']]
]

count = 50

for query_name, query in new_dict.items():
    print('Processing query:', query_name)

    query_perf_details = {}

    for spec in index_combinations:
        spec_name = util.build_index_description_key(all_indexes, spec) if spec else '__'

        print('Processing spec:', spec_name)

        for index in all_indexes:
            if index not in spec:
                mod_index = util.add_drop_index(db_eng, 'drop', index[0], index[1])

        for index in spec:
            mod_index = util.add_drop_index(db_eng, 'add', index[0], index[1])

        perf_details = util.calc_time_diff_per_year(db_eng, count, {query_name: query})

        query_perf_details[spec_name] = perf_details[query_name]

    all_perf_details[query_name] = query_perf_details
    print('Finished processing query:', query_name, '\n')

util.write_perf_data(all_perf_details, 'listings_join_reviews_test.json')


In [ ]:
  q5 = """
SELECT CAST(EXTRACT(YEAR FROM datetime) AS INTEGER) AS year, COUNT(*) AS review_count
FROM reviews
group by year;
"""
with db_eng.connect() as conn:
    result_reviews = conn.execute(sql_text(q5))
    print()
    print(result_reviews.all())
